In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from collections import namedtuple

import threading

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

from resize_generator import ResizeImageGenerator
from augmentation import ImageGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

def get_colored_data(images, coordinates):
    return (
        np.array([image for image in images if len(np.shape(image)) == 3]),
        np.array([answer for image, answer in zip(images, coordinates) if len(np.shape(image)) == 3])
    )

In [3]:
all_x, all_y, points_number = get_train_data()
colored_x, colored_y = get_colored_data(all_x, all_y)
input_shape=(100, 100, 3)
train_x, test_x, train_y, test_y = train_test_split(
    colored_x, colored_y, test_size=0.3, random_state=42
)

datagen = ImageGenerator()

In [4]:
generator = ResizeImageGenerator(input_shape)

In [5]:
regularization_lambda = 5e-3

def add_Conv2D_relu(model, n_filter, filters_size, input_shape=None):
    if input_shape is not None:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu', input_shape=input_shape,
        ))
    else:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu'
        ))

model = Sequential()

add_Conv2D_relu(model, 64, (3, 3), input_shape)
add_Conv2D_relu(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 128, (3, 3))
add_Conv2D_relu(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 256, (3, 3))
add_Conv2D_relu(model, 256, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten(name='flatten'))
model.add(Dense(
    512, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
))
model.add(Dropout(0.5))
model.add(Dense(
    512, activation='elu',
    kernel_regularizer=regularizers.l2(regularization_lambda)
))

model.add(Dense(points_number * 2))

In [6]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(decay=1e-3),
              metrics=['mse'])

In [7]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 0

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_mse:_' + str(logs['mean_squared_error']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [8]:
batch_size = 32
model.fit_generator(
    generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=1000,
    callbacks=[MyClallback(model, 'generator_models')],
    validation_data=generator.flow(test_x, test_y, batch_size=batch_size),
    validation_steps=int(len(test_y) / batch_size)
)

Epoch 1/1000
130/130 [==============================] - 42s - loss: 187.7576 - mean_squared_error: 173.9516 - val_loss: 188.6831 - val_mean_squared_error: 174.4263
Epoch 2/1000
130/130 [==============================] - 36s - loss: 68.5189 - mean_squared_error: 54.4205 - val_loss: 72.5520 - val_mean_squared_error: 58.6454
Epoch 3/1000
 18/130 [===>..........................] - ETA: 25s - loss: 57.6812 - mean_squared_error: 43.8007

KeyboardInterrupt: 